In [ ]:
pip install openai

In [ ]:
pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
openai_key = os.environ.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = openai_key
)

In [ ]:
from dataclasses import dataclass

@dataclass
class ExternalKnowledge:
    title: str
    content: str
    
    def __str__(self) -> str:
        return f"{self.title}: {self.content}"

wafa = """
    Muhammad Wafa Al Ausath, or commonly known as Wafa, is a student majoring in Informatics Engineering at Lampung University.
    Wafa has a strong interest in the field of information technology, especially mobile application development.
    He has experience as a mobile application developer at Coolva Technology and has worked as a freelance mobile developer.
    He has also served as a mobile developer intern at several startup companies.
    
    github profile = https://github.com/mikirinkode
    linkedin profile = www.linkedin.com/in/muhammadwafaalausath
    personal website = https://mikirinkode.com
""".strip()

extra_knowledges = [
    ExternalKnowledge(title="Wafa",content=wafa),
]

In [ ]:
import json
import datetime

class MyAssistant:
    def __init__(self, openai_client, extra_knowledges, name = "Miriko"):
        self.name = name
        self.messages = [
            {"role": "system", "content": f"""
             You are a Wafa's personal AI Assistant that helps him with his daily tasks. 
             Here some extra knowledges for you {extra_knowledges}.
             """.strip()},
        ]
        self.client = openai_client
        print(self.greet())

    def greet(self):
        # print(self.chat("Say Greeting"))
        return f"Hello, {self.name} here. Anything i could help today?"

    # promt -> input user
    # system_content -> role the ai that we want
    # model -> used gpt model
    # stream -> to display the result in real time (like typing)
    def chat(self, prompt, system_content = None, model = "gpt-3.5-turbo", stream = False):
        
        system_content = system_content if system_content != None else self._determine_system_content(prompt)
            
        self.messages.append({"role": "system", "content": system_content}) 
        
        self.messages.append({"role": "user", "content": prompt})
        
        if stream:
            completion = self.client.chat.completions.create(
                model = model,
                messages = self.messages,
                stream = True
            )
            for chunk in completion:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
        else: 
            completion = client.chat.completions.create(
                model = model,
                messages = self.messages
            )
            result = completion.choices[0].message.content
            self._handle_result(result)
            
            
    def _determine_system_content(self, content):
        if "summarize" in content:
            return self._summarizer()
        elif "translate" in content:
            return self._translator()
        elif "correct" in content:
            return self._english_grammar_master()
        elif "who is" in content:
            return self._who_is()
        elif "how to" in content:
            return self._how_to()
        else:
            return """
        You are a Wafa's personal AI Assistant that helps him with his daily tasks.
             
             You also have this following tools:
             Time () -> return current time
             Date () -> return current date
             
             Output in the following format:
             {
                 direct_answer: "<your answer", # if you have direct answer
                 invoke_external_tool: "<tool name>([parameter])", # if you want to invoke external tool
             }
        """
            
    def _handle_result(self, result):
        json_result = json.loads(result)
        
        if "direct_answer" in result:
            direct_answer = json_result.get("direct_answer")
            print(direct_answer)
        elif "invoke_external_tool" in result:
            tool_name = json_result.get("invoke_external_tool")
            if tool_name == "Time()":
                print(datetime.datetime.now().strftime("%H:%M"))
            elif tool_name == "Date()":
                print(datetime.datetime.now().strftime("%A, %d %B %Y"))
        else:
            print("Here is the result:")
            print(result)
    
    
    def _summarizer(self):
        return """
                        You are a summarizer AI that summarizes the given content in two sentences.
                        The output format should be in a list
                    """.strip()

    def _translator(self):
        return """
                        You are a translator AI that translates the given content to user's requested language.
                        Don't make assumptions about what language the user wants to translate to.
                        Ask for clarification if a user doesn't specify a language.
                    """.strip()
        
    def _english_grammar_master(self):
        return """You are an English grammar master that corrects the given content.
                    """.strip()

    def _who_is(self):
        return """if the user asks about a person, return in this JSON format:
                            {
                                "name": "John Doe", # must be a full name
                                "skills": ["Mobile Development", "Web Development", "Machine Learning"],
                                "experience": [ # The experience must be clear information, if there is some missing information use empty string
                                    {
                                        "company": "Coolva Technology",
                                        "role": "Mobile Developer",
                                        "duration": 1 # must be return in year
                                    },
                                    {
                                        "company": "Gojek",
                                        "role": "Software Engineer",
                                        "duration": 1 # must be return in year
                                    }
                                ]
                            }
                            
                            if the user ask about How To Do something, 
                         """.strip()
        
    def _how_to(self):
        return """please return in this JSON List format:
                            [
                                {
                                    "step" : "step name",
                                    "description": "step description",
                                    "tools" : ["tool 1", "tool 2"], # optional, if there is a tool that needed
                                    "difficulty": "" # can be easy, medium, hard, or empty string
                                }
                            ]
                    """.strip()
        
    def display_chat_history(self):
        for message in self.messages:
            print(message["role"], ":", message["content"])

In [ ]:
my_assistant = MyAssistant(openai_client=client, extra_knowledges=extra_knowledges)

In [ ]:
my_assistant.chat("do you know who am i?")

In [ ]:
my_assistant.chat("who is wafa?")

In [ ]:
my_assistant.chat("how to make an instant noodle?")

In [ ]:
my_assistant.chat("i want 'hello, how are you?' be translated in another language")

In [ ]:
my_assistant.chat("in japanese please")

In [ ]:
my_assistant.display_chat_history()

In [ ]:
my_assistant.chat("what time is it now?")

In [ ]:
my_assistant.chat("what date is it now?")

In [2]:
!python -m pip install semantic-kernel==0.4.3.dev0

  Obtaining dependency information for semantic-kernel==0.4.3.dev0 from https://files.pythonhosted.org/packages/14/a2/468d2caa9c14de4d9e2d9c6adb39e71d91a2ee874ecb525838fcf2c92784/semantic_kernel-0.4.3.dev0-py3-none-any.whl.metadata
  Obtaining dependency information for aiofiles<24.0.0,>=23.1.0 from https://files.pythonhosted.org/packages/c5/19/5af6804c4cc0fed83f47bff6e413a98a36618e7d40185cd36e69737f3b0e/aiofiles-23.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for motor<4.0.0,>=3.3.1 from https://files.pythonhosted.org/packages/3f/9a/1a43a329dffbd1a631c52e64c1e9c036621afdfd7f42096ae4bf2de4132b/motor-3.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for openai>=1.0 from https://files.pythonhosted.org/packages/e7/44/5ece9adb8b5943273c845a1e3200168b396f556051b7d2745995abf41584/openai-1.6.1-py3-none-any.whl.metadata
  Using cached openai-1.6.1-py3-none-any.whl.metadata (17 kB)
  Obtaining dependency information for openapi_core<0.19.0,>=0.18.0 from http

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.5.3 which is incompatible.
ypy-websocket 0.8.2 requires aiofiles<23,>=22.1.0, but you have aiofiles 23.2.1 which is incompatible.


In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()
kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

prompt = kernel.create_semantic_function("""
1) A robot may not injure a human being or, through inaction,
allow a human being to come to harm.

2) A robot must obey orders given it by human beings except where
such orders would conflict with the First Law.

3) A robot must protect its own existence as long as such protection
does not conflict with the First or Second Law.

Give me the TLDR in exactly 5 words.""")

print(prompt())


Robots must prioritize human safety.


In [4]:
skill = kernel.import_semantic_skill_from_directory("../../samples/skills", "FunSkill")
joke_function = skill["Joke"]

print(joke_function("time travel to dinosaur age"))

ValueError: Skill directory does not exist: FunSkill